# Match Endpoint Integration Tests

This notebook contains a series of integration tests for the Match endpoint of the UD DS API. It should be run from top-to-bottom using the Run All command.
The notebook should run completely without error, and if run inside the DS codebase will clean up after itself.

## Setup

In [4]:
# Uncomment this line if you have problems with import
# %cd ../

In [5]:
import requests
from data_generators.generators import RandomMentee, RandomMentor

In [6]:
url = 'http://127.0.0.1:8000'
create_url = f"{url}/create/match"
delete_url = f"{url}/delete/match"
read_url = f"{url}/read/match"

In [7]:
n_users = 2 # Number of mock users to create for each type (mentors and mentees)
mentors = []
mentees = []

for _ in range(n_users):
    mentor = vars(RandomMentor())
    mentor['other_info'] = "TEST"
    assert requests.post(f"{url}/create/mentor", json=mentor).json() == {"result": True}, "Issue creating mock mentors"
    mentors.append(mentor)

    mentee = vars(RandomMentee())
    mentee['other_info'] = "TEST"
    assert requests.post(f"{url}/create/mentee", json=mentee).json() == {"result": True}, "Issue creating mock mentees"
    mentees.append(mentee)

## Tests

### _Create a match with a new mentor_

Create a mentor/mentee match with mentor[0] and mentee[0]

In [8]:
mentor_id = mentors[0]['profile_id']
mentee_id = mentees[0]['profile_id']

ret = requests.post(create_url, json={"mentor_id": mentor_id, "mentee_id": mentee_id})

assert ret.status_code == 200, f"Unexpected status code {ret.status_code}"

expected = {"result": True}
assert ret.json() == expected, f"Unexpected return data {ret.json()}"

AssertionError: Unexpected status code 404

### _Get the profile information for the mentors of a specified mentee who has a single mentor._
Return the matched mentors for mentee[0]. We should get the profile information for mentor[0] as the only item in a list.

In [ ]:
query = {"user_id": mentee_id, "user_type": "mentee"}
actual = requests.post(read_url, json=query).json()
expected = {"result": [mentors[0]]}
actual['result'][0].pop('created_at')  # Remove created_at field for comparison purposes
assert actual == expected, 'Unexpected return for /read/matches'

### _Get the profile information for the mentees of a specified mentor who has a single mentee._
Return the matched mentors for mentor[0]. We should get the profile information for mentee[0] as the only item in a list.


In [ ]:
query = {"user_id": mentor_id, "user_type": "mentor"}
actual = requests.post(read_url, json=query).json()
expected = {"result": [mentees[0]]}
actual['result'][0].pop('created_at')  # Remove created_at field for comparison purposes
assert actual == expected, 'Unexpected return for /read/matches'

### _Add mentee to existing mentor_
Add a second mentee to our existing mentor[0]

In [ ]:
mentee_id = mentees[1]['profile_id']

ret = requests.post(create_url, json={"mentor_id": mentor_id, "mentee_id": mentee_id})

assert ret.status_code == 200, f"Unexpected status code {ret.status_code}"

expected = {"result": True}
assert ret.json() == expected, f"Unexpected return data {ret.json()}"

### _Read mentees for mentor with multiple mentees_
Get list of mentee profiles for mentor with multiple mentee matches.

In [ ]:
query = {"user_id": mentor_id, "user_type": "mentor"}
actual = requests.post(read_url, json=query).json()

for profile in actual['result']:
    profile.pop('created_at')  # Remove created_at field for comparison purposes

actual = sorted(actual['result'], key=lambda p: p['profile_id'])
expected = sorted(mentees[0:2], key=lambda p: p['profile_id'])
assert actual == expected, 'Unexpected return for /read/matches'

### _Create a second mentor with an existing mentee_
Create a new mentor[1] and assign them mentee[0]

In [ ]:
mentor_id = mentors[1]['profile_id']
mentee_id = mentees[0]['profile_id']

ret = requests.post(create_url, json={"mentor_id": mentor_id, "mentee_id": mentee_id})

assert ret.status_code == 200, f"Unexpected status code {ret.status_code}"

expected = {"result": True}
assert ret.json() == expected, f"Unexpected return data {ret.json()}"

### _Get mentors for mentee with multiple mentors_
Get list of profiles for the mentors of mentee[0]

In [ ]:
query = {"user_id": mentee_id, "user_type": "mentee"}
actual = requests.post(read_url, json=query).json()

for profile in actual['result']:
    profile.pop('created_at')  # Remove created_at field for comparison purposes

actual = sorted(actual['result'], key=lambda p: p['profile_id'])
expected = sorted(mentors[0:2], key=lambda p: p['profile_id'])
assert actual == expected, 'Unexpected return for /read/matches'

### _Delete match_
Delete one mentor/mentee match for mentor and mentee who each have multiple

In [ ]:
mentor_id = mentors[0]['profile_id']
mentee_id = mentees[0]['profile_id']

ret = requests.post(delete_url, json={"mentor_id": mentor_id, "mentee_id": mentee_id})

assert ret.status_code == 200, f"Unexpected status code {ret.status_code}"

expected = {"result": True}
assert ret.json() == expected, f"Unexpected return data {ret.json()}"

### _Check that delete was successful_
Verify that the previous delete call was successful. Mentor[0] should only have mentee[1] as a mentee, and mentee[0] should only have mentor[1] as a mentor.

In [ ]:
# Check list of mentees for mentor
query = {"user_id": mentor_id, "user_type": "mentor"}
actual = requests.post(read_url, json=query).json()
expected = {"result": [mentees[1]]}

for profile in actual['result']:
    profile.pop('created_at')  # Remove created_at field for comparison purposes

assert actual == expected, 'Unexpected return for /read/matches'

# Check list of mentors for mentee
query = {"user_id": mentee_id, "user_type": "mentee"}
actual = requests.post(read_url, json=query).json()
expected = {"result": [mentors[1]]}

for profile in actual['result']:
    profile.pop('created_at')  # Remove created_at field for comparison purposes

assert actual == expected, 'Unexpected return for /read/matches'

### _Delete only match for mentor and mentee_
Delete the match between mentor/mentee who have no other matches.

In [ ]:
mentor_id = mentors[0]['profile_id']
mentee_id = mentees[1]['profile_id']

ret = requests.post(delete_url, json={"mentor_id": mentor_id, "mentee_id": mentee_id})

assert ret.status_code == 200, f"Unexpected status code {ret.status_code}"

expected = {"result": True}
assert ret.json() == expected, f"Unexpected return data {ret.json()}"

### _Verify that delete was successful_
Verify that queries for mentor[0] and mentee[1] both return empty lists.

In [ ]:
# Check list of mentees for mentor
query = {"user_id": mentor_id, "user_type": "mentor"}
actual = requests.post(read_url, json=query).json()
expected = {"result": []}

for profile in actual['result']:
    profile.pop('created_at')  # Remove created_at field for comparison purposes

assert actual == expected, 'Unexpected return for /read/matches'

# Check list of mentors for mentee
query = {"user_id": mentee_id, "user_type": "mentee"}
actual = requests.post(read_url, json=query).json()
expected = {"result": []}

for profile in actual['result']:
    profile.pop('created_at')  # Remove created_at field for comparison purposes

assert actual == expected, 'Unexpected return for /read/matches'

## Cleanup
Remove all Mentor, Mentee, and Match entries that were created during the testing process

In [ ]:
from app.data import MongoDB

db = MongoDB()

for collection in ['Mentors', 'Mentees']:
    db.delete(collection, {'other_info': 'TEST'})

db.delete('Matches', {'mentor_id': {"$in": [mentor['profile_id'] for mentor in mentors]}})